In [1]:
#!pip install azure-mgmt-purview

In [2]:
import os
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
#from azure.identity import ClientSecretCredential 
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.purview import PurviewManagementClient
from azure.mgmt.purview.models import *
from datetime import datetime, timedelta
import time

In [3]:
 # Azure subscription ID
subscription_id = '0f5eaf2f-3dc4-45cb-b94c-bd95d29d99df'

# This program creates this resource group. If it's an existing resource group, comment out the code that creates the resource group
rg_name = 'purviewmuneesh-rg'

# The purview name. It must be globally unique.
purview_name = 'purviewmuneesh-ac'

In [4]:
credential = AzureCliCredential()  #login using az login

In [5]:
resource_client = ResourceManagementClient(credential, subscription_id)

In [6]:
purview_client = PurviewManagementClient(credential, subscription_id)

In [7]:
rg_result = resource_client.resource_groups.create_or_update(rg_name, {"location": "eastus"})

In [8]:
#Create a purview
identity = Identity(type= "SystemAssigned")
sku = AccountSku(name= 'Standard', capacity= 1)
purview_resource = Account(identity=identity,sku=sku,location ="eastus")

In [9]:
try:
    pa = (purview_client.accounts.begin_create_or_update(rg_name, purview_name, purview_resource)).result()
    print("location:", pa.location, " Microsoft Purview Account Name: ", purview_name, " Id: " , pa.id ," tags: " , pa.tags) 
except:
    print("Error in submitting job to create account")
    print_item(pa)
    
while (getattr(pa,'provisioning_state')) != "Succeeded" :
        pa = (purview_client.accounts.get(rg_name, purview_name))  
        print(getattr(pa,'provisioning_state'))
        if getattr(pa,'provisioning_state') == "Failed" :
            print("Error in creating Microsoft Purview account")
            break
        time.sleep(30)    

location: eastus  Microsoft Purview Account Name:  purviewmuneesh-ac  Id:  /subscriptions/0f5eaf2f-3dc4-45cb-b94c-bd95d29d99df/resourceGroups/purviewmuneesh-rg/providers/Microsoft.Purview/accounts/purviewmuneesh-ac  tags:  None
Creating
Creating
Creating
Creating
Creating
Creating
Succeeded


In [11]:
pa = purview_client.accounts.begin_delete(rg_name, purview_name).result()